## Setup sumo

In [1]:
import os
import sys
if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import traci

In [2]:
sys.path.append(os.path.join('c:', os.sep, 'whatever', 'path', 'to', 'sumo', 'tools'))


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import libsumo as traci

In [14]:
sumoBinary = os.getenv("sumoBinary")
sumoCmd = [sumoBinary, "-c", os.getenv("sumocfg")]

In [6]:
ds = {}
dict1 = {}
ds['v1'] = dict1 
print('v2' in ds)

False


## Create Car class

In [7]:
import traci._vehicle
import numpy as np


class veh_info:
    def __init__(self, name):
        self.name = name
        self.info()
        self.front = None
        self.rear = None
        self.orear = None
        self.ofront = None
        
    def info(self):
        self.speed_()
        self.posi_()
        self.lane_()
        self.acc_()
        self.angle_()

    def angle_(self):
        self.angle = traci.vehicle.getAngle(self.name)
    def speed_(self):
        self.speed = traci.vehicle.getSpeed(self.name)
    def posi_(self):
        self.posi = traci.vehicle.getPosition(self.name)
    def lane_(self):
        self.lane = traci.vehicle.getLaneIndex(self.name)
    def acc_(self):
        self.acc = traci.vehicle.getAcceleration(self.name)
    
    def difposi(self,other):
        a = np.array(self.posi)
        b = np.array(other.posi)
        dist = np.linalg.norm(a-b)
        return dist

    def near(self, listofobj):
        self.front = None
        self.rear = None
        frontposi = None
        rearposi = None
        for i in listofobj:
            if self != i:
                if self.lane == i.lane:
                    #check front or back
                    if self.projposi(i) > 1:
                        # print(self.name , "  ",self.projposi(i))           
                    ## for front
                        if frontposi == None:
                            fronttempfrontposi = self.difposi(i)
                            self.front = i
                        else:
                            if frontposi > fronttempfrontposi:
                                frontposi = fronttempfrontposi
                                self.front = i
                    ## for back
                    else:
                        # print(self.name , "  ",self.projposi(i)) 
                        if rearposi == None:
                            reartemprearposi = self.difposi(i)
                            self.rear = i
                        else:
                            if rearposi > reartemprearposi:
                                rearposi = reartemprearposi
                                self.rear = i                            

        # print(self.front)

    def nearOtherLane(self, lane_name, listofobj):
        frontposi = None
        rearposi = None
        self.orear = None
        self.ofront = None
        for i in listofobj:
            if self != i:
                if lane_name == i.lane:
                    #check front or back
                    # print(self.projposi(i))
                    if self.projposi(i) > 1:
                        # print(self.name , "  ",self.projposi(i))           
                    ## for front
                        if frontposi == None:
                            fronttempfrontposi = self.difposi(i)
                            self.ofront = i
                        else:
                            if frontposi > fronttempfrontposi:
                                frontposi = fronttempfrontposi
                                self.ofront = i
                    ## for back
                    else:
                        # print(self.name , "  ",self.projposi(i)) 
                        if rearposi == None:
                            reartemprearposi = self.difposi(i)
                            self.orear = i
                        else:
                            if rearposi > reartemprearposi:
                                rearposi = reartemprearposi
                                self.orear = i  
        # print(self.ofront)
    
    def projposi(self, other):
        n = np.array(self.posi)
        anglep = np.arctan2(n[1],n[0])* 180 / np.pi
        angle = self.angle
        n_norm = np.sqrt(sum(n**2))    
        u = np.array(other.posi)
        proj_of_u_on_n = (np.dot(u, n)/n_norm**2)
        if np.absolute(angle - anglep) > 90:    #find if car is traveling inreverse to the vector of the map  
            proj_of_u_on_n*-1     
        return proj_of_u_on_n
    
    def olanetoc(self,lane_name, listofobj):
        self.nearOtherLane(lane_name, listofobj)
        front = self.ofront
        rear = self.orear
        # print(rear, " ", front)
        fronttoc = None
        reartoc = None
        ##front
        if front != None:
            rx = self.difposi(front)
            rv = self.speed - front.speed
            ra = self.acc - front.acc
            
            if rx < 8:
                fronttoc = 0
            else:
                if ra == 0: # Handle zero acceleration
                    if rv != 0:
                        t = rx / rv
                        if t > 0:  # Only future collisions
                            fronttoc = t
                else:
                    discriminant = rv**2 + 2 * ra * rx
                    if discriminant >= 0:
                        sqrt_d = np.sqrt(discriminant)
                        t1 = (-rv + sqrt_d) / ra
                        t2 = (-rv - sqrt_d) / ra    
                        valid_times = [t for t in (t1, t2) if t > 0] # Find positive times
                        if valid_times:
                            fronttoc = min(valid_times)  # Earliest collision

        if rear != None:
            rx = self.difposi(rear)
            rv = -(self.speed - rear.speed)
            ra = -(self.acc - rear.acc)

            if rx < 8:
                reartoc = 0
            else:
                if ra == 0: # Handle zero acceleration
                    if rv != 0:
                        t = rx / rv
                        if t > 0:  # Only future collisions
                            reartoc = t
                else:
                    discriminant = rv**2 + 2 * ra * rx
                    if discriminant >= 0:
                        sqrt_d = np.sqrt(discriminant)
                        t1 = (-rv + sqrt_d) / ra
                        t2 = (-rv - sqrt_d) / ra
                        valid_times = [t for t in (t1, t2) if t > 0] # Find positive times
                        if valid_times:
                            reartoc = min(valid_times)  # Earliest collision

        return reartoc, fronttoc  
            # if ra == 0:
            #     reartoc = rx/rv
            # elif rv**2 - 2*ra*rx > 0:
            #     t1 = (np.sqrt(2* ra *rx + rv**2) - rv) /ra
            #     t2 = -(np.sqrt(2* ra *rx + rv**2) + rv) /ra
            #     if t1 < t2 and t1 > 0:
            #         reartoc = t1
            #     elif t2 > 0:
            #         reartoc = t2
            # else:
            #     reartoc = None
     
        return reartoc,fronttoc
    
    def lanetoc(self, listofobj):
        self.near(listofobj)
        front = self.front
        rear = self.rear
        # print(rear, " ", front)
        fronttoc = None
        reartoc = None
        ##front
        if front != None:
            rx = self.difposi(front)
            rv = self.speed - front.speed
            ra = self.acc - front.acc
            if rx < 8:
                fronttoc = 0
            else:
                if ra == 0: # Handle zero acceleration
                    if rv != 0:
                        t = rx / rv
                        if t > 0:  # Only future collisions
                            fronttoc = t
                else:
                    discriminant = rv**2 + 2 * ra * rx
                    if discriminant >= 0:
                        sqrt_d = np.sqrt(discriminant)
                        t1 = (-rv + sqrt_d) / ra
                        t2 = (-rv - sqrt_d) / ra    
                        valid_times = [t for t in (t1, t2) if t > 0] # Find positive times
                        if valid_times:
                            fronttoc = min(valid_times)  # Earliest collision

        if rear != None:
            rx = self.difposi(rear)
            rv = -(self.speed - rear.speed)
            ra = -(self.acc - rear.acc)
            if rx < 8:
                reartoc = 0
            else:
                if ra == 0: # Handle zero acceleration
                    if rv != 0:
                        t = rx / rv
                        if t > 0:  # Only future collisions
                            reartoc = t
                else:
                    discriminant = rv**2 + 2 * ra * rx
                    if discriminant >= 0:
                        sqrt_d = np.sqrt(discriminant)
                        t1 = (-rv + sqrt_d) / ra
                        t2 = (-rv - sqrt_d) / ra
                        valid_times = [t for t in (t1, t2) if t > 0] # Find positive times
                        if valid_times:
                            reartoc = min(valid_times)  # Earliest collision
        return reartoc, fronttoc  

In [8]:
def updateobject(listofobj,arrived_vehicles):
    listofobj.pop(arrived_vehicles[0], None)
    return listofobj

## Rou file Creating

In [9]:
import random
import xml.etree.ElementTree as ET

# Parameters
num_vehicles = 2  # Number of vehicles to generate
num_flows = 1      # Number of flows to generate
edges = ["E0"]  # List of possible edges

# Create XML structure
routes = ET.Element("routes", {
    "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
    "xsi:noNamespaceSchemaLocation": "http://sumo.dlr.de/xsd/routes_file.xsd"
})

# Define routes
for i, edge in enumerate(edges):
    ET.SubElement(routes, "route", {"id": f"r_{i}", "edges": edge})

# Define flows
for i in range(num_flows):
    ET.SubElement(routes, "flow", {
        "id": f"f_{i}",
        "begin": str(random.uniform(0, 5)),  # Random start time
        "end": "3600.00",
        "color": "255,255,255",
        "route": "r_0",
        "departSpeed": str(random.uniform(5, 6)),  # Random speed between 3 and 8 m/s
        "vehsPerHour": str(random.randint(1000, 2000)),  # Random vehicle flow rate
        "departLane": "1"
    })

# Define vehicles
ET.SubElement(routes, "vehicle", {
    "id": "v_0",
    "depart": str(random.uniform(10, 15)),  # Random depart time
    "departSpeed": str(random.uniform(3, 5)),  # Random speed between 3 and 8 m/s
    "color": "255,0,0",
    "route": "r_0"
})

ET.SubElement(routes, "vehicle", {
    "id": "v_1",
    "depart": str(random.uniform(18, 25)),  # Random depart time
    "departSpeed": str(random.uniform(3, 5)),  # Random speed between 3 and 8 m/s
    "color": "0,0,255",
    "route": "r_0"
})

# Convert to XML string
tree = ET.ElementTree(routes)
with open("generated_routes.rou.xml", "wb") as file:
    tree.write(file, encoding="utf-8", xml_declaration=True)

print("SUMO route file generated: generated_routes.rou.xml")


SUMO route file generated: generated_routes.rou.xml


## Sumo Sim

In [10]:
import traci._vehicle
import gc

traci.start(sumoCmd)
listdicts = {}
step = 0
listofobj = []
listoftoc = {}
listofcars = {}
lanechanged ={}
car_v_1 = None

while traci.simulation.getMinExpectedNumber() > 0:
    for veh_id in traci.vehicle.getIDList():
        try:
            if veh_id not in listdicts.keys(): 
                globals()["car_" + veh_id] = veh_info(veh_id)
                listdicts[veh_id] = globals()["car_" + veh_id]
            listofobj = list(listdicts.values())

            # listofobj = list(updateobject(listdicts,arrived_vehicles).values())

            globals()["car_" + veh_id].info()
            globals()["car_" + veh_id].near(listofobj)

            # Disable safety mechanisms

            traci.vehicle.setLaneChangeMode(veh_id, 0)
            traci.vehicle.setSpeedMode(veh_id, 0)

                # traci.vehicle.changeLane("v_0", 1, 1)

           
            arrived_vehicles = traci.simulation.getArrivedIDList()

            if arrived_vehicles:
                # print(f"Vehicles arrived this step: {arrived_vehicles[0]}")
                listofobj = list(updateobject(listdicts,arrived_vehicles).values())
        except traci.TraCIException as e:
            print(f"Error controlling vehicle {veh_id}: {e}")

        #####get info

    if "v_1" in list(listdicts.keys()):
        globals()["car_" + veh_id].info()
        car_v_1.near(listofobj)

        # Slow down
        if  car_v_1.front != None:
            if car_v_1.lanetoc(listofobj)[1]  != None and car_v_1.lanetoc(listofobj)[1] < 6:
                traci.vehicle.setSpeed("v_1", car_v_1.front.speed)

        # LaneChangeChecking
        car_v_1.nearOtherLane(1,listofobj)
        listoftoc[step] = car_v_1.olanetoc(1,listofobj)
        listofcars[step] = [car_v_1.orear,car_v_1.ofront]         
        if len(list(listoftoc)) > 6:
            if car_v_0 in listofobj:
                traci.vehicle.setSpeed("v_0", 2)

        if len(list(listoftoc)) > 8:
            if all([i is None or i > 5 for s in range(step-2,step+1) for i in listoftoc[s]]):
                traci.vehicle.changeLane("v_1", 1, 1)
                lanechanged[step] = step
                if  car_v_1.front != None:
                    traci.vehicle.setSpeed("v_1", car_v_1.front.speed)
                    

    # Log collisions
    collisions = traci.simulation.getCollidingVehiclesIDList()
    if collisions:
        print(f"Step {step}: Collisions detected: {collisions}")
        print(collisions[0])
    traci.simulationStep()
    step += 1

traci.close()


FileNotFoundError: [WinError 2] The system cannot find the file specified

### Check TTC

In [ ]:
lanechanged

{38: 38, 39: 39, 43: 43, 47: 47, 51: 51, 52: 52}

In [ ]:
step = list(lanechanged.keys())[0]

In [ ]:
for i in range(step-4,step+4):
    print(f"step {i} , ttc {listoftoc[i]}")

step 34 , ttc (np.float64(26.705563649584203), None)
step 35 , ttc (np.float64(7.679386987889853), 0)
step 36 , ttc (np.float64(5.841301078281949), None)
step 37 , ttc (np.float64(28.541066461815976), None)
step 38 , ttc (np.float64(7.498151789622734), None)
step 39 , ttc (np.float64(7.0242705746157705), None)
step 40 , ttc (None, np.float64(1.4916910849510707))
step 41 , ttc (None, None)


In [ ]:
[i is None or i > 5 for s in range(step-2,step+1) for i in listoftoc[s]]

[np.True_, True, np.True_, True, np.True_, True]

In [ ]:
all([i is None or i > 3 for s in range(step-2,step+1) for i in listoftoc[s]])

True

In [ ]:
# ((listdicts['v_0'])[1])[0]